# Fast Tree of DADA2 ASVs
* Tree building from alignment using DADA2 sequences from Exp10.  
* Fasta has already been thresholded to reduce computational demand
* Use qiime environment and set kernel to Python 2

In [1]:
workDir = '/home/be68/Hyphosphere/Exp10/DADA2/'
OTUFileDir = '/home/be68/Hyphosphere/Exp10/DADA2/'
OTUFile = 'seqs_thresh.fasta'


nprocs = 3

In [2]:
import os
import numpy as np
#import entrez.direct
from cogent.app.fasttree import build_tree_from_alignment
from cogent import DNA, LoadSeqs
from Bio import Entrez
Entrez.email = "be68@cornell.edu"
#from IPython.display import display, Image, SVG
#from cogent3 import LoadSeqs, FastTree


In [3]:
#%load_ext rpy2.ipython

In [4]:
if not os.path.isdir(workDir):
    os.mkdir(workDir)

In [5]:
%cd $workDir

/data/home/be68/Hyphosphere/Exp10/DADA2


In [6]:
!cd $workDir; ln -f -s $OTUFileDir$OTUFile

ln: ‘/home/be68/Hyphosphere/Exp10/DADA2/seqs_thresh.fasta’ and ‘./seqs_thresh.fasta’ are the same file


In [7]:
!cd $workDir; head $OTUFile

>ASV1
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGACTATTAAGTCAGGGGTGAAATCCCGGGGCTCAACCCCGGAACTGCCTTTGATACTGGTAGTCTTGAGTTCGAGAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAACACCAGTGGCGAAGGCGGCTCACTGGCTCGATACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAGG
>ASV2
GACATAGGTGGCAAACATTATCCGGAATTATTGGGCGTAAAGGGTGCGTAGGCGGCATGATAAGTTGCTGGTGGGAAATCAAGGCTCAACCTTGTGGAAGCTAGCAATACTGTCAAGCTAGAGGGCAGAAGAGGTTAACGGAACTCTATGTGGAGCGGTAAAATGTGTAGATATATAGAAGAACATCAATAAAGGCGAAGGCAGTTAACTAGTCTGTCCCTGACGTTGAGGCACGAAAGCGTGGGGAGCAAAACGG
>ASV3
TACGGAGGGAGCTAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCACGTAGGCGGCTTTGTAAGTCAGAGGTGAAAGCCTGGAGCTCAACTCCAGAACTGCCTTTGAGACTGCATCGCTTGAATCCAGGAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAAGAACACCAGTGGCGAAGGCGGCTCACTGGACTGGTATTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAGG
>ASV4
GACGGGGGGGGCAAGTGTTCTTCGGAATGACTGGGCGTAAAGGGCACGTAGGCGGTGAATCGGGTTGAAAGTGAAAGTCGCCAAAAAGTGGCGGAATGCTCTCGAAACCAATTCACTTGAGTGAGACAGAGGAGAGTGGAATTTCGTGTGTAGGGGTGAAATCCGTAGATCTACGAAGGAACGCCAAAAGCGAAGGCAGCTCTCTGGGTCC

In [8]:
!printf "Number of OTUs in fasta: "
!cd $workDir; grep -c ">" $OTUFile

Number of OTUs in fasta: 8162


## Using SSU-Align to align seqs and masking based on alignment posterior probabilities.

In [9]:
!cd $workDir; ssu-prep -f -x -b 50 --rfonly --dna $OTUFile ssu_aln 3

# _ssu-prep :: prepare SSU rRNA sequences for parallel ssu-align jobs
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-prep -x -f -b 50 --dna --rfonly seqs_thresh.fasta ssu_aln 3
# date:    Tue Dec 11 15:05:42 2018
#
# Validating input sequence file ... done.
#
# Preparing 3 ssu-align jobs ...
# Partitioning seqs with goal of equalizing total number of nucleotides per job ...
#
# output file name      description                                           
# --------------------  ------------------------------------------------------
  ssu_aln/seqs_thresh.fasta.1  partition 1 FASTA sequence file (2721 seqs; 688666 nt)
  ssu_aln/seqs_thresh.fasta.2  partition 2 FASTA sequence file (2721 seqs; 688707 nt)
  ssu_aln/seqs_thresh.fasta.3  partition 3 FASTA sequence file (2720 seqs; 688487 nt)
  ssu_aln.ssu-align.sh  

In [10]:
!cd $workDir; ./ssu_aln.ssu-align.sh

# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/seqs_thresh.fasta.1 ssu_aln/ssu_aln.1 > /dev/null &
# Executing: ssu-align -b 50 --dna --rfonly ssu_aln/seqs_thresh.fasta.2 ssu_aln/ssu_aln.2 > /dev/null &
# Executing: ssu-align --merge 3 -b 50 --dna --rfonly ssu_aln/seqs_thresh.fasta.3 ssu_aln/ssu_aln.3
# _ssu-align :: align SSU rRNA sequences
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-align --merge 3 -b 50 --dna --rfonly ssu_aln/seqs_thresh.fasta.3 ssu_aln/ssu_aln.3
# date:    Tue Dec 11 15:05:49 2018
#
# Validating input sequence file ... done.
#
# Stage 1: Determining SSU start/end positions and best-matching models...
#
# output file name            description                                
# --------------------------  -------------------------------------------
  ssu_aln.3.tab               lo

In [11]:
!cd $workDir; ssu-mask --dna --afa ssu_aln

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask --afa --dna ssu_aln
# date:    Tue Dec 11 15:19:09 2018
#
# Masking alignments based on posterior probabilities...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  ssu_aln.archaea.stk         input   aln   1508      -      -
  ssu_aln.archaea.mask       output  mask   1508    246   1262
  ssu_aln.archaea.mask.pdf   output   pdf   1508    246   1262
  ssu_aln.archaea.mask.afa   output   aln    246      -      -
#
  ssu_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask      output  mask

In [ ]:
#!cd $workDir; ~/seq_tools/seq_tools fasta_unwrap ssu_aln/ssu_aln.bacteria.mask.afa  | head

### Using Sulfolobus as the root (acc. X90478).

In [12]:
sso_acc = "X90478"
sso_fa = Entrez.efetch(db="nucleotide", id=sso_acc, rettype="fasta", retmode="text").readlines()

In [13]:
out = open(os.path.join(workDir, 'X90478.fasta'), 'w')
sso_fa_namestrip = sso_fa[1:]
sso_fa_namestrip.insert(0,">%s\n"%sso_acc)
out.writelines(sso_fa_namestrip)
out.close()

In [14]:
!cd $workDir; head -n 4 X90478.fasta

>X90478
TCCTGCCGGTCCCGACCGCTATCGGGGTGGGGCTAAGCCATGGGAGTCGTACGCTCCCGGGCAAGGGAGC
GTGGCGGACGGCTGAGTAACACGTGGCTAACCTACCCTGAGGAGGGAGATAACCCCGGGAAACTGGGGAT
AATCTCCCATAGGCGAGGAGTCCTGGAACGGTTCCTCGCTGAAAGGCTCATGGGCTATTCCCCGCTCATG


### Align the outgroup to the same cm (use the same mask that was established earlier).

In [15]:
!cd $workDir; ssu-align -f -n bacteria --dna --rfonly X90478.fasta sso_aln

# _ssu-align :: align SSU rRNA sequences
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-align -f -n bacteria --dna --rfonly X90478.fasta sso_aln
# date:    Tue Dec 11 15:20:16 2018
#
# Validating input sequence file ... done.
#
# Stage 1: Determining SSU start/end positions and best-matching models...
#
# output file name          description                                
# ------------------------  -------------------------------------------
  sso_aln.tab               locations/scores of hits defined by HMM(s)
  sso_aln.bacteria.hitlist  list of sequences to align with bacteria CM
  sso_aln.bacteria.fa             1 sequence  to align with bacteria CM
#
# Stage 2: Aligning each sequence to its best-matching model...
#
# output file name          description
# ------------------------  -------------------

In [16]:
!cd $workDir; ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --dna --afa sso_aln/

# _ssu-mask :: mask SSU rRNA alignments
# SSU-ALIGN 0.1.1 (Feb 2016)
# Copyright (C) 2016 Howard Hughes Medical Institute
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# command: _ssu-mask -s ssu_aln/ssu_aln.bacteria.mask --afa --dna sso_aln/
# date:    Tue Dec 11 15:20:23 2018
#
# Masking alignments using pre-existing masks...
#
#                                                     mask    
#                                                 ------------
# file name                  in/out  type  #cols  incl.  excl.
# -------------------------  ------  ----  -----  -----  -----
  sso_aln.bacteria.stk        input   aln   1582      -      -
  ssu_aln.bacteria.mask       input  mask   1582    248   1334
  sso_aln.bacteria.mask.pdf  output   pdf   1582    248   1334
  sso_aln.bacteria.mask.afa  output   aln    248      -      -
#
# All attempts to draw structure diagrams of masks were successful.
#
# List o

In [ ]:
#!cd $workDir; /home/bryan/seq_tools/seq_tools fasta_unwrap sso_aln/sso_aln.bacteria.mask.afa | head

In [17]:
!cd $workDir; cat sso_aln/sso_aln.bacteria.mask.afa ssu_aln/ssu_aln.bacteria.mask.afa > aln_for_tree.fasta

In [19]:
#!cd $workDir; /home/bryan/seq_tools/seq_tools fasta_unwrap aln_for_tree.fasta | head

/bin/sh: 1: /home/bryan/seq_tools/seq_tools: not found


In [18]:
!cd $workDir; head aln_for_tree.fasta

>X90478
TACCAGCCCCGCGAG---TGGTCGGGATTACTGGGCCTAAAGCGCCCGTAGCCGGCCCGA
CAAGTCACTCCTTAAAGACCCCGGCTCAACCGGGGGAAGGGTGATACTGTCGGGCTAGGG
GGCGGGAGAGGCCAGCGGTACTCCCGGAGTAGGGGCGAAATCCTCAGATCTCGGGAGGAC
CACCAGTGGCGAAAGCGGCTGGCTAGAACGCCCGACGGTGAGGGGCGAAAGCCGGGGCAG
CAAAAGGG
>ASV1
TACGAAGGGGGCTAGCGTTGTTCGGATTTACTGGGCGTAAAGCGCACGTAGGCGGACTAT
TAAGTCAGGGGTGAAATCCCGGGGCTCAACCCCGGAACCTTTGATACTGGTAGTCTTGAG
TTCGAGAGAGGTGAGTGGAATTCCGAGTGTAGAGGTGAAATTCGTAGATATTCGGAGGAA


## Inferring and rooting the tree

In [19]:
aln = LoadSeqs(os.path.join(workDir, 'aln_for_tree.fasta'), moltype=DNA)
t_unroot = build_tree_from_alignment(aln, moltype=DNA)

In [20]:
t_rooted = t_unroot.rootedWithTip(sso_acc)

In [21]:
t_rooted.writeToFile(os.path.join(workDir, 'ASV.tree'))